In [ ]:
import numpy as np

# Pandas ne ajuta sa citim fisiere .xlsx
import pandas as pd
import re
import random

import transformers as trans
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
from tensorflow.keras.models import load_model
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
flan_t5_tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')
bert_tokenizer = AutoTokenizer.from_pretrained("readerbench/RoBERT-base")

In [ ]:
bert_2 = load_model('/kaggle/input/good-demo-data-set/bert_2.h5', custom_objects={"TFBertModel": trans.TFBertModel})

In [ ]:
bert_5 = load_model('/kaggle/input/good-demo-data-set/bert_5.h5', custom_objects={"TFBertModel": trans.TFBertModel})

In [ ]:
flan_t5_2 = TFAutoModelForSeq2SeqLM.from_pretrained('/kaggle/input/good-demo-data-set/flan_t5_2/kaggle/working/flan_t5_2')

In [ ]:
flan_t5_5 = TFAutoModelForSeq2SeqLM.from_pretrained('/kaggle/input/good-demo-data-set/flan_t5_5_version3/kaggle/working/flant_t5_5_version3')

In [ ]:
review = "Daca mai merg pe pedale in picioare (cand urc spre exemplu, nu mereu) sunt probleme cu butucul pedalier."

In [ ]:
task_prefix_5 = "Score the following review on a scale from 1 to 5:"
task_prefix_2 = "Score the following review on a scale from 0 to 1:"

In [ ]:
bert_encoded_review = bert_tokenizer(review, padding=True, return_tensors="np")
flan_t5_5_encoded_review = flan_t5_tokenizer(task_prefix_5 + review, max_length=512, truncation=True, return_tensors="tf")
flan_t5_2_encoded_review = flan_t5_tokenizer(task_prefix_2 + review, max_length=512, truncation=True, return_tensors="tf")

In [ ]:
bert_input = [bert_encoded_review['input_ids'], 
         bert_encoded_review['token_type_ids'],
         bert_encoded_review['attention_mask']]

In [ ]:
bert_5_prediction = bert_5.predict(bert_input)
bert_2_prediction = bert_2.predict(bert_input)

In [ ]:
print(bert_5_prediction)
print(bert_2_prediction)

bert_rating_5 = tf.argmax(bert_5_prediction, axis=1)
bert_rating_5 = bert_rating_5 + 1
bert_rating_5 = bert_rating_5.numpy()

bert_rating_2 = tf.argmax(bert_2_prediction, axis=1)
bert_rating_2 = bert_rating_2
bert_rating_2 = bert_rating_2.numpy()

print(bert_rating_5)
print(bert_rating_2)

In [ ]:
def clean_review(review):
    # strip dash but keep a space
    review = review.replace('-', ' ')
    # lower case the data
    review = review.lower()
    # remove excess spaces near punctuation
    #df['body'] = df['body'].apply(lambda x: re.sub(r'\s([?.!"](?:\s|$))', r'\1', x))
    # remove excess white spaces
    review = review.strip()
    # add " </s>" to end of body
    review = review + " </s>"
    
    return review

In [ ]:
review = clean_review(review)

flan_t5_5_prediction = flan_t5_5.generate(flan_t5_5_encoded_review.input_ids, max_length=2)
flan_t5_5_prediction = flan_t5_tokenizer.decode(flan_t5_5_prediction[0], skip_special_tokens=True)

In [ ]:
flan_t5_2_prediction = flan_t5_2.generate(flan_t5_2_encoded_review.input_ids, max_length=2)
flan_t5_2_prediction = flan_t5_tokenizer.decode(flan_t5_2_prediction[0], skip_special_tokens=True)

if flan_t5_2_prediction == ' ':
    print(1)
else:
    print(0)
    
print(flan_t5_5_prediction)